In [1]:
import AFQ.data as afqd
import cloudknot as ck
import importlib
import s3fs

/Users/richford/miniconda3/envs/cloudknot_qsiprep/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/richford/miniconda3/envs/cloudknot_qsiprep/lib/python3.7/site-packages/dask/dataframe/utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/Users/richford/miniconda3/envs/cloudknot_qsiprep/lib/python3.7/site-packages/dipy/stats/__init__.py:7: UserWarning: The `dipy.stats` module is still under heavy development and functionality, as well as the API is likely to change in future versions of the software
  warnings.warn(w_string)
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials f

In [55]:
cc = afqd.S3BIDSStudy(
    study_id="camcan",
    bucket="cam-can.afq-insight",
    s3_prefix="raw/cc700/mri/pipeline/release004/BIDS_NRDG/BIDS_NRDG",
    subjects="all",
    anon=False,
)

/Users/richford/miniconda3/envs/cloudknot_qsiprep/lib/python3.7/site-packages/bids/layout/models.py:102: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  FutureWarning)


Retrieving subject S3 keys
[########################################] | 100% Completed |  1min  8.5s


In [56]:
len(cc.subjects)

657

In [45]:
ck.get_region()

'us-west-2'

In [46]:
ck.set_region('us-west-2')

In [47]:
fs = s3fs.S3FileSystem()
fs.touch("cam-can.afq-insight/raw/cc700/mri/pipeline/release004/BIDS_NRDG/BIDS_NRDG/derivatives/qsiprep-ck/useless_file")

## Define the preprocessing function

In [48]:
def preprocess_camcan_subject(subject_id):
    import AFQ.data as afqd
    import os
    import subprocess
    import sys
    from bids import BIDSLayout
    from s3fs import S3FileSystem

    cc = afqd.S3BIDSStudy(
        study_id="camcan",
        bucket="cam-can.afq-insight",
        s3_prefix="raw/cc700/mri/pipeline/release004/BIDS_NRDG/BIDS_NRDG",
        subjects=subject_id,
        anon=False,
    )
    
    cc.download("./cam-can")
    fs = S3FileSystem()
    
    for non_sub_s3_key in cc.non_sub_s3_keys["raw"]:
        if "tmp_dcm2bids" not in non_sub_s3_key:
            fs.get(non_sub_s3_key,
                   os.path.join("./cam-can", os.path.basename(non_sub_s3_key)))

    response = subprocess.run(
        [
            "qsiprep",
            "--output-resolution",
            "1.8",
            "--unringing-method",
            "mrdegibbs",
            "--force-spatial-normalization",
            "--participant-label",
            subject_id,
            "-w",
            "./cc-wrk",
            "--nthreads",
            "8",
            "--omp-nthreads",
            "8",
            "./cam-can",
            "./cam-can-preproc",
            "participant",
        ],
        check=True
    )
    
    output_dir = "cam-can.afq-insight/raw/cc700/mri/pipeline/release004/BIDS_NRDG/BIDS_NRDG/derivatives/qsiprep-ck"            
    fs.put(f"./cam-can-preproc/qsiprep/{subject_id}",
           "/".join([output_dir, subject_id]), recursive=True)
    fs.put(f"./cam-can-preproc/qsiprep/{subject_id}.html",
           "/".join([output_dir, subject_id + ".html"]))

    return {subject_id: True}

# Create a cloudknot DockerImage instance

where the base_image is a customized version of qsiprep
This version of the qsiprep docker image is located on AWS ECR with image URI:
[454929164628.dkr.ecr.us-east-1.amazonaws.com/qsiprep:nrdg](https://console.aws.amazon.com/ecr/repositories/qsiprep/?region=us-east-1)

In [49]:
di = ck.DockerImage(
    name="preprocess-camcan-subject",
    func=preprocess_camcan_subject,
    base_image="qsiprep:nrdg",
    github_installs=["https://github.com/richford/pyAFQ.git@s3-bids-fetch",
                     "https://github.com/bids-standard/pybids.git@0.9.3"],
    overwrite=True
)

## Build, tag, and push the Docker image

In [50]:
di.build(tags=["camcan-preproc-20200803"])

In [51]:
repo = ck.aws.DockerRepo(name=ck.get_ecr_repo())

In [52]:
repo.repo_uri

'454929164628.dkr.ecr.us-west-2.amazonaws.com/cloudknot'

In [53]:
# The very first time you run this, this command could take
# a few hours because the docker image is large
di.push(repo=repo)

## Create the Knot

In [81]:
sub_ids = [s.subject_id for s in cc.subjects]

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
chunk_knots = {
    n: {
        # Specify bid_percentage to use Spot instances
        # And make sure the volume size is large enough. 50-55 GB seems about right for HBN preprocessing. YMMV.
        "knot": ck.Knot(
            name=f"cc_preproc_20200804_ch{n}",
            docker_image=di,
            pars_policies=("AmazonS3FullAccess", "AllowCamCanBucketAccess"),
            bid_percentage=100,
            memory=64000,
            job_def_vcpus=8,
            volume_size=55
        ),
        "subjects": chunk
    } for n, chunk in enumerate(chunks(sub_ids, 33))
}

KeyboardInterrupt: 

## Submit the jobs and check on results

In [73]:
import numpy as np

In [74]:
dwi_predict = {
    sub.subject_id: any(["dwi" in s3key for s3key in sub.s3_keys["raw"]])
    for sub in cc.subjects
}

In [77]:
print(np.sum(np.logical_not(np.array(list(dwi_predict.values())))))
print(np.sum(np.array(list(dwi_predict.values()))))

15
642


In [61]:
for i in chunk_knots.keys():
    chunk_knots[i] = {
        "knot": chunk_knots[i]["knot"],
        "subjects": chunk_knots[i]["subjects"],
        "results": chunk_knots[i]["knot"].map(chunk_knots[i]["subjects"])
    }

In [80]:
array_props = {key: 0 for key in chunk_knots[0]["knot"].jobs[0].status["arrayProperties"]["statusSummary"]}
for _knot in chunk_knots.values():
    for prop in array_props.keys():
        array_props[prop] += _knot["knot"].jobs[0].status["arrayProperties"]["statusSummary"][prop]
        
array_props

{'STARTING': 0,
 'FAILED': 19,
 'RUNNING': 0,
 'SUCCEEDED': 638,
 'RUNNABLE': 0,
 'SUBMITTED': 0,
 'PENDING': 0}

## Results

The results are dicts where the keys are the subject IDs and the values report sucess or failure

In [ ]:
result_futures.result()

## Figure out how many subjects we have in the entire study to support some cost estimates

In [ ]:
all_sites = {}

for site in ["Site-SI", "Site-CBIC", "Site-RU", "Site-CUNY"]:
    all_sites[site] = afqd.HBNSite(site=site)
    print(f"{site}: {len(all_sites[site]._all_subjects)}")

## When you're done, clobber the knot

In [ ]:
knot.clobber(clobber_pars=True)